# Creating a proto-dataset

proto-dataset is the daily to which is added per record the base. This Jupyter Notebook is optimized for myDRE Workspace using OSDS template.

Use **configuration.xlsx** to change the variables.

Upload Castor export as zip to Workspace, choose as target: ./EDC.

If the zip ends up in transfer (so not in ./EDC), the script will deal with it.

In [1]:
import pandas as pd
import shutil
import glob
import generic_functions2 as gf

In [2]:
# Settings

file = 'CASTOR_configuration_proto.xlsx'
sheet_name='Settings'
settings_table = pd.read_excel(file, sheet_name=sheet_name, index_col='Item')

edc_folder = settings_table.loc['edc_folder']['Value']
proto_dataset_folder = settings_table.loc['proto_dataset_folder']['Value']
study_name = settings_table.loc['study_name']['Value']
recordid = settings_table.loc['recordid']['Value']
study_daily = study_name + settings_table.loc['study_daily']['Value']
study_base = study_name + settings_table.loc['study_base']['Value']
separator = settings_table.loc['separator']['Value']

In [3]:
# When 

# move Castor zips from inbox when manually uploaded to a Workspace
source_folders = ['z:/inbox/', 'i:/inbox/']
# source_folder = 'z:/inbox/'
for source_folder in source_folders:
    try:
        gf.move_files(study_name+'*.*', source_folder, edc_folder)
    except:
        pass

#extract potential zip files
gf.find_unpack_zips(edc_folder, 6)

# gf.print_title('Choose the source for the Study')
study_dir = gf.choose_dir_item(edc_folder,'folders',study_name)
study_daily = edc_folder + study_dir + '/' + study_daily  + study_dir[-9:] + '.csv'
study_base = edc_folder + study_dir + '/' + study_base  + study_dir[-9:] + '.csv'

print('\n'*2)\n
# This part is to remove unnecessary columns, this speeds up the processing considerably
# The assumption is that what doesn't need to be mapped, isn't necessary
print('*'*30)
print('Please select the conversion remapping file that will be used')
print('This will be used to identify which variables can be dropped from the source')
conversion_excel = gf.choose_dir_item('./','files','xlsx')

print('\n'*2)
gf.print_title('Processing, please relax and maybe grab a cup of coffee/tea ....')

./EDC/
******************************************************************************************
Choose source by number
------------------------------------------------------------------------------------------
[1]  EraCORE_csv_export_20210929	[1]  EraCORE_csv_export_20210929	[2]  EraCORE_csv_export_20210907	
[3]  EraCORE_csv_export_20210831	



Please choose Q(uit) or between 1 and 4:  1


In [ ]:
# creating the pandas
df_castor_daily = pd.read_csv(study_daily, sep=separator, dtype='string')
df_castor_base = pd.read_csv(study_base, sep=separator, dtype='string')

df_castor_daily.sort_values([recordid], ascending=True, inplace=True)
df_castor_base.sort_values([recordid], ascending=True, inplace=True)

# creating list with base and daily
study_dfs = [df_castor_daily, df_castor_base]


# creating the list with variables that will be used
sheet_name = 'Conversion_Table'
index_col = 'Map_Variable'
df_variables_mapped = pd.read_excel(conversion_excel, sheet_name=sheet_name, index_col=index_col)
variables_used = set(df_variables_mapped.index.values.tolist())

# creating the list with IDs that must be dropped
sheet_name = 'Remove_IDs'
index_col = 'Remove_IDs'
df_remove_ids = pd.read_excel(conversion_excel, sheet_name=sheet_name, index_col=index_col)
df_remove_ids.index = df_remove_ids.index.map(str)
remove_ids = set(df_remove_ids.index.values.tolist())

# remove all 'unnecessary variables'
for df in study_dfs:
    print(f'pre:  has dimensions {df.shape}')
#     df = df[df.columns.intersection(variables_used)]
    print(f'post: has dimensions {df.shape}')
print('\\n'*2)

# remove the IDs that must be dropped
for df in study_dfs:
    df = df[~df[recordid].isin(remove_ids)]




In [5]:
# remove all 'unnecessary variables'
for df in study_dfs:
    df.drop(columns=[col for col in df if col not in variable_used], inplace = True)

# remove the IDs that must be dropped
for df in study_dfs:
    df = df[~df[recordid].isin(remove_ids)]

# Add base to daily for eacht participant-day
## create list of all base variables except Record Id
base_variables = list(df_castor_base.columns)
daily_variables = list(df_castor_daily.columns)

## list with variables to remove
remove_base_variables = [recordid, 'Unnamed: 758']

## remove the variables
base_variables = list(set(base_variables)-set(remove_base_variables))

#check and if necessary rename base variable when it exists in daily
for item in base_variables:
    if item in daily_variables:
        df_castor_base.rename(columns={item: 'DUB_'+item}, inplace=True)

## copy df_eracore_daily to df_eracore and add base_variables to df_eracore
df_castor = df_castor_daily.copy()
df_castor[base_variables] = ''

In [ ]:
## add base values to the right records
def add_base_values(Id):
    try:
        df_castor.loc[df_castor[recordid]==Id, base_variables] = \
        df_castor_base.loc[df_castor_base[recordid]==Id, base_variables].values
        return Id
    except:
        pass

temp = [add_base_values(Id) for Id in df_castor_base[recordid]]
for Id in df_castor_base[recordid]:
    add_base_values(Id)


## remove base and daily from memory
del df_castor_daily
del df_castor_base
del study_dfs

In [6]:
# Saving the prot-Castor both as .sx.sx as csv
output_path = './proto-dataset'
df_castor.to_excel(output_path+ '/proto-dataset.xlsx', index=False)
gf.output_csv(df_castor, output_path+'/proto-dataset', False)

input('Enter to continue....')
print('\n'*2)
gf.print_title('Finished creating Proto-'+study_name)

saved: ./proto-Castor/Proto-Castor.csv



******************************************************************************************
Finished creating Proto-EraCORE
------------------------------------------------------------------------------------------
